In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
from datetime import timedelta

In [5]:
technicals=pd.read_hdf('technicals.h5')

In [6]:
#code in cell eases matrix indexing
def returnRSI(RSI):
    if 0<=RSI<20:
        return 0
    elif 20<=RSI<40:
        return 1
    elif 40<=RSI<60:
        return 2
    elif 60<=RSI<80:
        return 3
    else:
        return 4
#accepted range for MACD is -150 to 150
def returnMACD(MACD):
    if MACD<-90:
        return 0
    elif -90<MACD<-30:
        return 1
    elif -30<MACD<30:
        return 2
    elif 30<MACD<90:
        return 3
    else:
        return 4
#return traders cumulative profits in terms of positive and negative
def returnProfit(Profit):
    if Profit>=0:
        return 1
    else:
        return 0
#return current position in terms of long, short or flat
def returnPosition(Position):
    if Position>0:
        return 1
    elif Position==0:
        return 0
    else:
        return -1

In [7]:
#data partition
length=int(0.8*len(technicals))
temp=technicals.reset_index()
test=temp[temp.index<=length].set_index(['index'])
train=temp[temp.index>length].set_index(['index'])

In [8]:
nanless=test.dropna()
start_RSI=returnRSI(nanless.RSI[0])
start_MACD=returnMACD(nanless.MACD[0])

In [9]:
start_date=nanless.head().index[0]
end_date=nanless.tail(1).index[0]

In [10]:
#our nanless frame will basically become a sparsified verion of the technicals frame
nanless['StateRSI']=nanless['RSI'].apply(lambda x: returnRSI(x))
nanless['StateMACD']=nanless['MACD'].apply(lambda x: returnMACD(x))

C:\Users\Padraic\Anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Padraic\Anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#define amount of buy or sell
test=0
temp=0
lot_size=1

In [ ]:
#define Q table, each value of Q has a different possible actions
#Q[s]=[long_val,hold_val,short_val]
#Q=3x2x5x5x3
q = np.ndarray(shape=(3,2,5,5,3),dtype=float)
q.fill(np.inf)

In [ ]:
class PnL:
    def __init__(self):
        #tracks current amount of symbol
        self.quantity = 0
        #cumulative sum of positions (net position)
        self.running_quantity=0
        #total negative from buying positions
        self.cost = 0.0
        #current market value of positions
        self.market_value = 0.0
        #realized pnl
        self.r_pnl = 0.0
        #average fill price of all positions
        self.average_price = 0.0
        #realized plus unrealized pnl is total pnl (ie what we want)
        self.total_pnl=0.0
    #function that processes fill statements
    def fill(self, n_pos, exec_price):
        #position delta from new buys
        pos_change = n_pos - self.quantity
        #buy or sell
        direction = np.sign(pos_change)
        #previous positioning
        prev_direction = np.sign(self.quantity)
        #quantity we end up with
        qty_closing = min(abs(self.quantity), abs(pos_change)) * direction if prev_direction != direction else 0
        #quantity we started with
        qty_opening = pos_change if prev_direction == direction else pos_change - qty_closing
        
        #updated cost with new positions
        new_cost = self.cost + qty_opening * exec_price
        #if the old position wasnt zero, modify cost for new amount *avg price
        #same for realized pnl
        if self.quantity != 0:
            new_cost += qty_closing * self.cost / self.quantity
            self.r_pnl += qty_closing * (self.cost / self.quantity - exec_price)

        #update current positioning, total positioning, cost
        self.quantity = n_pos
        self.running_quantity=n_pos+self.running_quantity
        self.cost = new_cost

    #update for market price and publish total_pnl
    def update(self, price):
        if self.quantity != 0:
            self.average_price = self.cost / self.quantity
        else: 
            self.average_price = 0
        self.market_value = self.quantity * price
        upl=self.market_value - self.cost
        self.total_pnl=upl+self.r_pnl
        return self.total_pnl

In [ ]:
kappa=1e-4
gamma=0.999
alpha=0.001
epsilon=0.1 #run without initially, then with
converge=1e-3
#initial delta
delta = np.inf
actionlots = [0,lot_size,-lot_size]

In [ ]:
#sparsify nanless so it runs faster
nanless=nanless[['close','StateRSI','StateMACD']]
#multiple yearly runs until convergence
while delta>converge:
    btc=PnL()
    pastpnl=0
    pnl=0
    position=[]
    observedRSIs=[]
    observedMACDs=[]
    cumpos=0
    states=[]
    #Run=pd.DataFrame(index=technicals.index,columns=['state','pnl','position','price'])
    #Run=Run.apply(transition,axis=1)
    #^experimental code to run with apply instead of iterrrows
    #initialize initial state w/0 PnL and 0 positions, initial 
    states.append([0,1,start_RSI,start_MACD])
    state = states[0]
    for i, row in nanless.iterrows():
        current_price=row['close']
        next_row=nanless[nanless.index==i+timedelta(seconds=1)]
        action = np.argmax(q[state])
        chpos = actionlots[action]
        cumpos += chpos
        position.append(chpos)
        if position!=0:
            #if we are taking a new position, we need to assign rewards
            btc.fill(position[-1],current_price)
            new_price=next_row['close']
            pnl=btc.update(new_price)[0]
            delta_v=pastpnl-pnl
            reward=delta_v-0.5*kappa*delta_v**2
        pastpnl=pnl
        # define next state and append to state list
        prevstate = state
        prevstate.append(action)
        p=returnProfit(pnl)
        discrete_position=returnPosition(btc.running_quantity)
        nextrsi = next_row['StateRSI']
        nextmacd = next_row['StateMACD']
        state = [p,discrete_position,nextrsi,nextmacd]
        states.append(state)
        if q[tuple(prevstate)] == np.inf: # prevstate is state,action
            prevq = 0
        else:
            prevq = q[tuple(prevstate)]
        if np.max(q[tuple(state)]) == np.inf:
            thisq = 0
        else:
            thisq = np.max(q[tuple(state)])
        wholeqprev = q
        q[tuple(prevstate)] = prevq + alpha*(reward+gamma*thisq-prevq)
        delta = np.max(np.subtract(q,wholeqprev))
    break

In [11]:
#prevq + alpha*(reward+gamma*thisq-prevq)
len(nanless.drop_duplicates())

24592999

In [ ]:
delta

In [ ]:
df = pd.DataFrame([[i] for i in range(10)], columns=['num'])

In [ ]:
def powers(x):
    return x, x**2, x**3, x**4, x**5, x**6
df['p1'], df['p2'], df['p3'], df['p4'], df['p5'], df['p6'] =zip(*df.apply(lambda x: powers(x)))

In [ ]:
df